In [3]:
# -*- coding: utf-8 -*-
"""
# 📊 SRC TRADING – Detector de Patrones de Velas Japonesas
---
Este notebook de Python está diseñado para ayudar a los traders e inversores a identificar automáticamente patrones clásicos de velas japonesas en datos de precios de acciones o criptomonedas.
Las velas japonesas son una forma de representar los movimientos del precio de un activo en un período de tiempo, mostrando los precios de apertura, cierre, máximo y mínimo. Los patrones formados por una o más velas son utilizados en el análisis técnico para predecir posibles movimientos futuros del mercado.

A continuación, sigue los pasos para:
1. Instalar las librerías necesarias.
2. Descargar los datos de un activo específico.
3. Visualizar el gráfico de velas.
4. Detectar e interpretar los patrones de velas.
"""

# In[0]:
# Cabecera visual
#
# Carga y muestra el logo de SRC TRADING
from IPython.display import display, Image, clear_output
import ipywidgets as widgets

header_html = """
<style>
    .header-container {
        display: flex;
        align-items: center;
        background-color: #1a1a1a;
        padding: 20px;
        border-radius: 10px;
        margin-bottom: 20px;
    }
    .header-text h1 {
        color: #fff;
        font-family: sans-serif;
        font-size: 2.5em;
        margin: 0;
        padding-left: 20px;
    }
    .header-text p {
        color: #bbb;
        font-family: sans-serif;
        font-size: 1em;
        margin: 5px 0 0 20px;
    }
</style>
<div class="header-container">
    <img src="https://placehold.co/100x100/1a1a1a/fff?text=SRC" alt="SRC TRADING Logo" style="border-radius: 8px;">
    <div class="header-text">
        <h1>SRC TRADING</h1>
        <p>Detector de Patrones de Velas Japonesas</p>
    </div>
</div>
"""
display(widgets.HTML(header_html))

# In[1]:
# 1. Instalación de librerías
#
# Ejecuta esta celda para instalar todas las dependencias necesarias.
# Las librerías que usaremos son:
# - `yfinance`: Para descargar datos financieros de Yahoo Finance.
# - `plotly`: Para crear gráficos interactivos.
# - `ipywidgets`: Para crear la interfaz de usuario interactiva en el notebook.
# - `pandas`: Para el manejo de datos en DataFrames.
# - `openpyxl`: Para exportar a archivos de Excel.
# - `fpdf`: Para generar archivos PDF.

!pip install yfinance plotly pandas ipywidgets openpyxl fpdf --quiet

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, FileLink
import numpy as np
import plotly.graph_objects as go
import io
from fpdf import FPDF

# In[2]:
# 2. Descarga de datos
#
# Utiliza los controles a continuación para ingresar el símbolo de la acción o criptomoneda y el rango de fechas para descargar los datos.
# Ejemplos de símbolos: `AAPL` (Apple), `TSLA` (Tesla), `BTC-USD` (Bitcoin).

# Configuración de widgets para la interfaz de usuario
style = {'description_width': 'initial'}
ticker_input = widgets.Text(
    value='BTC-USD',
    description='Símbolo del activo:',
    placeholder='Ej: AAPL, TSLA, BTC-USD',
    style=style
)
start_date_picker = widgets.DatePicker(
    description='Fecha de inicio:',
    value=(datetime.now() - timedelta(days=180)).date(),
    disabled=False,
    style=style
)
end_date_picker = widgets.DatePicker(
    description='Fecha de fin:',
    value=datetime.now().date(),
    disabled=False,
    style=style
)
# -- INICIO DEL CAMBIO: Selector de marco de tiempo
timeframe_selector = widgets.Dropdown(
    options=['1d', '1h', '15m', '5m', '1m'],
    value='1d',
    description='Marco de Tiempo:',
    style=style
)
# -- FIN DEL CAMBIO --
download_button = widgets.Button(
    description='Descargar Datos',
    button_style='primary',
    icon='download'
)
output = widgets.Output()

def download_data(b):
    with output:
        clear_output(wait=True)
        ticker = ticker_input.value
        start_date = start_date_picker.value
        end_date = end_date_picker.value
        timeframe = timeframe_selector.value

        if start_date >= end_date:
            print("¡Error! La fecha de inicio debe ser anterior a la fecha de fin.")
            return

        print(f"Descargando datos para {ticker} desde {start_date} hasta {end_date} con un intervalo de {timeframe}...")
        try:
            global df
            df = yf.download(ticker, start=start_date, end=end_date, interval=timeframe)
            if df.empty:
                print(f"¡Error! No se encontraron datos para el símbolo '{ticker}' en el rango de fechas seleccionado. Por favor, verifica el símbolo e intenta de nuevo.")
            else:
                print("¡Descarga completa! Primeras 5 filas del DataFrame:")
                display(df.head())
        except Exception as e:
            print(f"Ocurrió un error durante la descarga: {e}")

download_button.on_click(download_data)

# Muestra los widgets en el notebook
print("Ingrese el símbolo y el rango de fechas:")
# -- INICIO DEL CAMBIO: Mostrar el nuevo selector
display(ticker_input, start_date_picker, end_date_picker, timeframe_selector, download_button, output)
# -- FIN DEL CAMBIO --

# In[3]:
# 3. Visualización de velas
#
# Esta función grafica el chart de velas japonesas y el volumen de forma interactiva usando Plotly.

def graficar_velas(df, patterns_df=None):
    """
    Crea un gráfico interactivo de velas japonesas con volumen y patrones usando Plotly.
    """
    if df.empty:
        print("El DataFrame está vacío. Por favor, descarga los datos primero.")
        return

    # Crear la figura principal
    fig = go.Figure()

    # Añadir el gráfico de velas
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='Velas',
            increasing_line_color='#1b5e20',  # Verde
            decreasing_line_color='#d32f2f'   # Rojo
        )
    )

    # Añadir los patrones como trazas de puntos
    if patterns_df is not None and not patterns_df.empty:
        unique_patterns = patterns_df['pattern'].unique()
        for pattern_name in unique_patterns:
            pat_data = patterns_df[patterns_df['pattern'] == pattern_name]

            # Usar un símbolo diferente para los patrones para distinguirlos
            marker_symbol = 'triangle-up'
            if 'Hammer' in pattern_name or 'Doji' in pattern_name:
                marker_symbol = 'circle'
            elif 'Star' in pattern_name:
                marker_symbol = 'star'

            fig.add_trace(
                go.Scatter(
                    x=pat_data['date'],
                    y=pat_data['value'],
                    mode='markers',
                    marker=dict(
                        symbol=marker_symbol,
                        size=10,
                        color=pat_data.iloc[0]['color'],
                        line=dict(width=1, color='black')
                    ),
                    name=pattern_name,
                    hoverinfo='text',
                    hovertext=[f'Patrón: {p}' for p in pat_data['pattern']]
                )
            )

    # Añadir el gráfico de volumen
    fig.add_trace(
        go.Bar(
            x=df.index,
            y=df['Volume'],
            name='Volumen',
            marker_color='gray',
            yaxis='y2' # Asignar a un segundo eje Y
        )
    )

    # Actualizar el layout para los dos gráficos
    fig.update_layout(
        title=f'Gráfico de velas para {ticker_input.value}',
        xaxis_rangeslider_visible=False, # Ocultar el control deslizante de rango
        xaxis_title='Fecha',
        yaxis_title='Precio',
        yaxis2=dict(
            title='Volumen',
            overlaying='y',
            side='right',
            showgrid=False
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        template='plotly_dark' # Un tema oscuro para mejor visualización
    )

    print("Generando gráfico interactivo...")
    fig.show()

# In[4]:
# 4. Detección de patrones
#
# Aquí se implementa la lógica para detectar los patrones de velas clásicos.
# Cada función revisa una vela específica y sus predecesoras según la definición del patrón.

def es_doji(open_price, close_price, high, low):
    """
    Un Doji se forma cuando los precios de apertura y cierre son prácticamente idénticos.
    Esto indica indecisión en el mercado.
    """
    return np.isclose(open_price, close_price, rtol=0.01) and (high - low) > (high - close_price) * 1.5 and (high - low) > (open_price - low) * 1.5

def es_hammer(open_price, close_price, high, low):
    """
    El Martillo (Hammer) es un patrón alcista. Tiene un cuerpo pequeño en la parte superior del rango
    y una mecha inferior larga, indicando que los compradores rechazaron el precio bajo.
    """
    body_size = abs(close_price - open_price)
    lower_shadow = min(open_price, close_price) - low
    upper_shadow = high - max(open_price, close_price)
    return (lower_shadow >= 2 * body_size) and (upper_shadow < body_size)

def es_shooting_star(open_price, close_price, high, low):
    """
    La Estrella Fugaz (Shooting Star) es un patrón bajista. Tiene un cuerpo pequeño en la parte
    inferior del rango y una mecha superior larga, indicando que los vendedores rechazaron el precio alto.
    """
    body_size = abs(close_price - open_price)
    upper_shadow = high - max(open_price, close_price)
    lower_shadow = min(open_price, close_price) - low
    return (upper_shadow >= 2 * body_size) and (lower_shadow < body_size)

def es_engulfing(prev_open, prev_close, open_price, close_price):
    """
    El patrón envolvente (Engulfing) se forma cuando una vela grande
    envuelve completamente el cuerpo de la vela anterior.
    """
    bullish = (prev_close > prev_open) and (close_price > open_price) and (open_price < prev_close) and (close_price > prev_close) and ((open_price < prev_open) or (close_price > prev_close))
    bearish = (prev_close < prev_open) and (close_price < open_price) and (open_price > prev_close) and (close_price < prev_close) and ((open_price > prev_open) or (close_price < prev_close))
    return bullish, bearish

def es_harami(prev_open, prev_close, open_price, close_price):
    """
    El patrón Harami se forma cuando una vela pequeña está contenida dentro
    del cuerpo de la vela anterior.
    """
    bullish = (prev_close < prev_open) and (close_price > open_price) and (open_price > prev_close) and (close_price < prev_open)
    bearish = (prev_close > prev_open) and (close_price < open_price) and (open_price < prev_close) and (close_price > prev_open)
    return bullish, bearish

def es_spinning_top(open_price, close_price, high, low):
    """
    Un Spinning Top tiene un cuerpo pequeño y mechas largas en ambos lados.
    Indica indecisión.
    """
    body_size = abs(close_price - open_price)
    full_range = high - low
    upper_shadow = high - max(open_price, close_price)
    lower_shadow = min(open_price, close_price) - low
    return (body_size < 0.3 * full_range) and (upper_shadow > body_size) and (lower_shadow > body_size)

def es_dragonfly_doji(open_price, close_price, high, low):
    """
    Un Doji libélula (Dragonfly Doji) tiene una mecha inferior larga y el cuerpo en la parte
    superior. Es un patrón alcista.
    """
    return np.isclose(open_price, close_price, rtol=0.01) and np.isclose(open_price, high, rtol=0.01) and (open_price - low) > (high - open_price) * 3

def es_gravestone_doji(open_price, close_price, high, low):
    """
    Un Doji lápida (Gravestone Doji) tiene una mecha superior larga y el cuerpo
    en la parte inferior. Es un patrón bajista.
    """
    return np.isclose(open_price, close_price, rtol=0.01) and np.isclose(open_price, low, rtol=0.01) and (high - open_price) > (open_price - low) * 3

def es_marubozu(open_price, close_price, high, low):
    """
    Un Marubozu es una vela con un cuerpo muy grande y sin mechas.
    Indica un movimiento fuerte en una dirección.
    """
    full_range = high - low
    body_size = abs(close_price - open_price)
    # Marubozu alcista
    bullish = (close_price > open_price) and (body_size > 0.9 * full_range) and (low == open_price) and (high == close_price)
    # Marubozu bajista
    bearish = (close_price < open_price) and (body_size > 0.9 * full_range) and (low == close_price) and (high == open_price)
    return bullish, bearish


def detectar_patrones(df):
    """
    Función principal para iterar sobre el DataFrame y detectar todos los patrones.
    Retorna un DataFrame con las fechas y los patrones encontrados.
    """
    detected_patterns = []

    # Asegurarse de que el DataFrame tenga un índice de fecha
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # Convertir a numpy arrays para mejor rendimiento
    opens = df['Open'].values
    highs = df['High'].values
    lows = df['Low'].values
    closes = df['Close'].values

    for i in range(1, len(df)):
        open_price = opens[i]
        close_price = closes[i]
        high = highs[i]
        low = lows[i]

        prev_open = opens[i-1]
        prev_close = closes[i-1]

        # 1. Patrones de una vela
        if es_doji(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Doji', 'value': close_price, 'color': 'yellow'})

        if es_hammer(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Martillo (Hammer)', 'value': low, 'color': 'blue'})

        if es_shooting_star(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Estrella Fugaz (Shooting Star)', 'value': high, 'color': 'purple'})

        is_marubozu_bullish, is_marubozu_bearish = es_marubozu(open_price, close_price, high, low)
        if is_marubozu_bullish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Marubozu Alcista', 'value': close_price, 'color': 'green'})
        if is_marubozu_bearish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Marubozu Bajista', 'value': close_price, 'color': 'red'})

        if es_dragonfly_doji(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Doji Libélula', 'value': low, 'color': 'cyan'})

        if es_gravestone_doji(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Doji Lápida', 'value': high, 'color': 'magenta'})

        # 2. Patrones de dos velas
        is_engulfing_bullish, is_engulfing_bearish = es_engulfing(prev_open, prev_close, open_price, close_price)
        if is_engulfing_bullish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Engulfing Alcista', 'value': low, 'color': 'darkgreen'})
        if is_engulfing_bearish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Engulfing Bajista', 'value': high, 'color': 'darkred'})

        is_harami_bullish, is_harami_bearish = es_harami(prev_open, prev_close, open_price, close_price)
        if is_harami_bullish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Harami Alcista', 'value': low, 'color': 'olive'})
        if is_harami_bearish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Harami Bajista', 'value': high, 'color': 'maroon'})

        if es_spinning_top(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Peonza (Spinning Top)', 'value': close_price, 'color': 'orange'})

    return pd.DataFrame(detected_patterns)


# In[5]:
# 5. Resultados e interpretación
#
# Esta sección muestra los patrones detectados y permite visualizarlos en el gráfico.

def mostrar_resultados():
    """
    Función para mostrar los resultados de la detección y la interfaz de usuario.
    """
    if 'df' not in globals() or df.empty:
        print("No hay datos cargados. Por favor, descarga los datos primero en la sección 2.")
        return

    if len(df) < 2:
        print("No hay suficientes datos para detectar patrones. Se necesitan al menos dos velas.")
        return

    global patterns_df
    patterns_df = detectar_patrones(df.copy())

    print("\n### Patrones detectados:")
    if patterns_df.empty:
        print("No se encontraron patrones en el rango de fechas seleccionado.")
        return

    # Muestra una tabla con las últimas 10 detecciones
    print("Últimas 10 detecciones de patrones:")
    display(patterns_df.tail(10))

    # Botones para descargar resultados
    download_csv_button = widgets.Button(
        description='Descargar CSV',
        button_style='info',
        icon='save'
    )
    download_excel_button = widgets.Button(
        description='Descargar Excel',
        button_style='info',
        icon='save'
    )
    download_pdf_button = widgets.Button(
        description='Descargar PDF',
        button_style='info',
        icon='save'
    )

    def download_csv(b):
        csv_output = patterns_df.to_csv(index=False)
        display(FileLink('patterns_detected.csv', data=csv_output, result_html_prefix="Click para descargar el archivo:"))

    def download_excel(b):
        patterns_df.to_excel('patterns_detected.xlsx', index=False)
        display(FileLink('patterns_detected.xlsx', result_html_prefix="Click para descargar el archivo:"))

    def download_pdf(b):
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt="Reporte de Patrones de Velas Japonesas", ln=True, align='C')
        pdf.ln(10)

        # Agregar la tabla de resultados
        pdf.set_font("Arial", size=10)
        # Encabezado
        for col in patterns_df.columns:
            pdf.cell(40, 7, col, 1, 0, 'C')
        pdf.ln()
        # Datos
        for index, row in patterns_df.iterrows():
            pdf.cell(40, 7, str(row['date'].strftime('%Y-%m-%d')), 1, 0)
            pdf.cell(40, 7, row['pattern'], 1, 0)
            pdf.cell(40, 7, str(round(row['value'], 2)), 1, 0)
            pdf.cell(40, 7, row['color'], 1, 0)
            pdf.ln()

        pdf_output = pdf.output(dest='S').encode('latin-1')
        with open('patterns_detected.pdf', 'wb') as f:
            f.write(pdf_output)
        display(FileLink('patterns_detected.pdf', result_html_prefix="Click para descargar el archivo:"))

    download_csv_button.on_click(download_csv)
    download_excel_button.on_click(download_excel)
    download_pdf_button.on_click(download_pdf)

    print("\n---")
    print("Selecciona los patrones que deseas ver en el gráfico:")

    # Widgets para filtrar patrones en el gráfico
    all_patterns = patterns_df['pattern'].unique()
    checkboxes = [widgets.Checkbox(description=p, value=True) for p in all_patterns]
    checkbox_container = widgets.VBox(checkboxes)
    display(checkbox_container)

    plot_button = widgets.Button(
        description='Generar Gráfico con Patrones',
        button_style='success',
        icon='bar-chart'
    )

    def plot_with_patterns(b):
        with output:
            clear_output(wait=True)
            selected_patterns = [cb.description for cb in checkboxes if cb.value]
            if not selected_patterns:
                print("Por favor, selecciona al menos un patrón para graficar.")
                return

            # Filtrar los patrones para el gráfico
            filtered_patterns = patterns_df[patterns_df['pattern'].isin(selected_patterns)]

            # Llamar a la función de graficado con los patrones filtrados
            graficar_velas(df, patterns_df=filtered_patterns)

    plot_button.on_click(plot_with_patterns)

    # Mostrar la interfaz de usuario con los botones de descarga
    display(plot_button, widgets.HBox([download_csv_button, download_excel_button, download_pdf_button]), output)

# Ejecutar la función para mostrar la interfaz de resultados
# Es necesario haber descargado los datos en la sección 2 antes de ejecutar esto
if 'df' in locals() and not df.empty:
    mostrar_resultados()


# In[6]:
# 6. Backtest de Patrones
#
# Simula operaciones de compra (largas) basadas en la aparición de un patrón específico.
#
# Este backtest es "ligero" y funciona de la siguiente manera:
# - Se asume una operación larga por cada vez que el patrón seleccionado aparece.
# - La entrada se realiza al precio de cierre de la vela donde se detecta el patrón.
# - La salida se realiza al precio de cierre del `Período de tenencia` después de la entrada.
#
# El backtest no considera comisiones, deslizamiento (slippage), ni otras consideraciones de trading real.

def backtest_patron(df_original, patterns_df, patron_seleccionado, periodo_tenencia):
    """
    Realiza un backtest simple para un patrón de velas específico.
    """
    # Filtrar patrones para el backtest
    trades = patterns_df[patterns_df['pattern'] == patron_seleccionado]

    if trades.empty:
        return {'total_trades': 0, 'winning_trades': 0, 'total_return': 0, 'max_drawdown': 0}

    capital_curve = [1000] # Empezamos con un capital base de 1000
    profits = []

    # Asegurarse de que el DataFrame de datos de velas esté ordenado
    df_original = df_original.sort_index()

    for date_found in trades['date']:
        try:
            # Encontrar el índice del patrón en el DataFrame original
            idx_entrada = df_original.index.get_loc(date_found)
            idx_salida = idx_entrada + periodo_tenencia

            # Asegurarse de que la operación de salida esté dentro de los datos
            if idx_salida < len(df_original):
                price_entrada = df_original.iloc[idx_entrada]['Close']
                price_salida = df_original.iloc[idx_salida]['Close']

                # Calcular el retorno del trade
                retorno_trade = (price_salida - price_entrada) / price_entrada

                # Actualizar las métricas
                profits.append(retorno_trade)

                # Actualizar el capital para calcular el drawdown
                ultimo_capital = capital_curve[-1]
                nuevo_capital = ultimo_capital * (1 + retorno_trade)
                capital_curve.append(nuevo_capital)

        except KeyError:
            # En caso de que la fecha no se encuentre en el DataFrame de datos
            continue

    # Calcular métricas finales
    total_trades = len(profits)
    winning_trades = sum(1 for p in profits if p > 0)

    total_return = (capital_curve[-1] - capital_curve[0]) / capital_curve[0] if len(capital_curve) > 1 else 0

    # Calcular el máximo drawdown
    capital_series = pd.Series(capital_curve)
    rolling_max = capital_series.expanding(min_periods=1).max()
    drawdowns = (capital_series / rolling_max) - 1
    max_drawdown = abs(drawdowns.min()) if not drawdowns.empty else 0

    return {
        'total_trades': total_trades,
        'winning_trades': winning_trades,
        'total_return': total_return,
        'max_drawdown': max_drawdown
    }

# Widgets para el backtest
backtest_output = widgets.Output()

def run_backtest(b):
    with backtest_output:
        clear_output(wait=True)
        if 'df' not in globals() or df.empty or 'patterns_df' not in globals() or patterns_df.empty:
            print("Por favor, primero descarga los datos y detecta los patrones en las secciones anteriores.")
            return

        selected_pattern = pattern_dropdown.value
        hold_period = hold_period_input.value

        if not selected_pattern:
            print("Por favor, selecciona un patrón para hacer el backtest.")
            return

        print(f"Realizando backtest para el patrón '{selected_pattern}' con un período de tenencia de {hold_period} velas...")

        # Filtrar patrones alcistas para backtest largos (compras)
        patrones_alcistas = ['Martillo (Hammer)', 'Doji Libélula', 'Marubozu Alcista', 'Engulfing Alcista', 'Harami Alcista']
        is_bullish = selected_pattern in patrones_alcistas

        # Si el patrón no es alcista, no se realiza un backtest "largo"
        if not is_bullish:
             print("Este backtest ligero solo simula operaciones de compra (largas). Por favor, selecciona un patrón alcista.")
             return

        # Backtest
        resultados = backtest_patron(df.copy(), patterns_df, selected_pattern, hold_period)

        # Mostrar resultados
        if resultados['total_trades'] > 0:
            win_rate = (resultados['winning_trades'] / resultados['total_trades']) * 100
            print("\n### Resultados del Backtest:")
            print(f"Patrón analizado: {selected_pattern}")
            print(f"Total de operaciones: {resultados['total_trades']}")
            print(f"Tasa de ganancia (Win Rate): {win_rate:.2f}%")
            print(f"Retorno total (ROI): {(resultados['total_return'] * 100):.2f}%")
            print(f"Máximo Drawdown: {(resultados['max_drawdown'] * 100):.2f}%")
        else:
            print("No se encontraron operaciones para este patrón. Por favor, revisa tus selecciones.")

# Creación de los widgets de backtest
pattern_dropdown_options = list(patterns_df['pattern'].unique()) if 'patterns_df' in globals() and not patterns_df.empty else ['No hay patrones detectados']
pattern_dropdown = widgets.Dropdown(
    options=pattern_dropdown_options,
    description='Patrón a Backtest:',
    disabled=('patterns_df' not in globals() or patterns_df.empty),
    style=style
)
hold_period_input = widgets.IntText(
    value=5,
    description='Período de tenencia (velas):',
    min=1,
    style=style
)
backtest_button = widgets.Button(
    description='Ejecutar Backtest',
    button_style='success',
    icon='play'
)

backtest_button.on_click(run_backtest)

print("Selecciona un patrón alcista y el período de tenencia para simular el rendimiento histórico.")
display(pattern_dropdown, hold_period_input, backtest_button, backtest_output)


# In[7]:
# 7. Estadísticas de Fiabilidad por Patrón
#
# Mide la eficacia empírica de los patrones de velas japonesas en tu conjunto de datos.
# Puedes definir una subida o bajada porcentual objetivo y el número de velas necesarias para alcanzarlo.

def calcular_fiabilidad(df_original, patterns_df, porcentaje_cambio, velas_adelante):
    """
    Calcula la fiabilidad de los patrones para un cambio porcentual y un período de tiempo dados.
    """
    resultados_fiabilidad = {}

    # Listas de patrones alcistas y bajistas para saber la dirección esperada
    patrones_alcistas = ['Martillo (Hammer)', 'Doji Libélula', 'Marubozu Alcista', 'Engulfing Alcista', 'Harami Alcista']
    patrones_bajistas = ['Estrella Fugaz (Shooting Star)', 'Doji Lápida', 'Marubozu Bajista', 'Engulfing Bajista', 'Harami Bajista']

    unique_patterns = patterns_df['pattern'].unique()

    for pattern in unique_patterns:
        # Ignorar patrones de indecisión para esta métrica de dirección
        if pattern not in patrones_alcistas and pattern not in patrones_bajistas:
            continue

        detecciones = patterns_df[patterns_df['pattern'] == pattern]
        total_detecciones = len(detecciones)
        detecciones_exitosas = 0

        if total_detecciones == 0:
            continue

        is_bullish = pattern in patrones_alcistas

        for date_found in detecciones['date']:
            try:
                # Encontrar el índice de la vela de detección
                idx_actual = df_original.index.get_loc(date_found)

                # Asegurarse de que haya suficientes velas para el período de análisis
                if idx_actual + velas_adelante < len(df_original):
                    precio_actual = df_original.iloc[idx_actual]['Close']
                    precio_futuro = df_original.iloc[idx_actual + velas_adelante]['Close']

                    cambio = (precio_futuro - precio_actual) / precio_actual

                    # Comprobar el éxito según la dirección del patrón
                    if is_bullish and cambio >= porcentaje_cambio / 100:
                        detecciones_exitosas += 1
                    elif not is_bullish and cambio <= -porcentaje_cambio / 100:
                        detecciones_exitosas += 1

            except KeyError:
                continue

        # Guardar los resultados
        tasa_exito = (detecciones_exitosas / total_detecciones) * 100 if total_detecciones > 0 else 0
        resultados_fiabilidad[pattern] = {
            'Total de Detecciones': total_detecciones,
            'Detecciones Exitosas': detecciones_exitosas,
            'Tasa de Fiabilidad (%)': round(tasa_exito, 2)
        }

    return pd.DataFrame.from_dict(resultados_fiabilidad, orient='index')

# Widgets para las estadísticas de fiabilidad
stats_output = widgets.Output()

def run_stats(b):
    with stats_output:
        clear_output(wait=True)
        if 'df' not in globals() or df.empty or 'patterns_df' not in globals() or patterns_df.empty:
            print("Por favor, primero descarga los datos y detecta los patrones en las secciones anteriores.")
            return

        porcentaje_cambio = change_percentage_input.value
        velas_adelante = periods_input.value

        if porcentaje_cambio <= 0 or velas_adelante <= 0:
            print("El porcentaje de cambio y el número de velas deben ser mayores que cero.")
            return

        print(f"Calculando fiabilidad para un cambio de {porcentaje_cambio}% en {velas_adelante} velas...")

        resultados = calcular_fiabilidad(df.copy(), patterns_df.copy(), porcentaje_cambio, velas_adelante)

        if resultados.empty:
            print("No se encontraron patrones o no se pudo calcular la fiabilidad.")
        else:
            print("\n### Resultados de Fiabilidad por Patrón:")
            display(resultados.sort_index())

# Creación de los widgets de fiabilidad
change_percentage_input = widgets.FloatText(
    value=2.0,
    description='Cambio porcentual (%):',
    min=0.1,
    style=style
)
periods_input = widgets.IntText(
    value=10,
    description='Período (velas):',
    min=1,
    style=style
)
calculate_button = widgets.Button(
    description='Calcular Fiabilidad',
    button_style='info',
    icon='check'
)

calculate_button.on_click(run_stats)

print("Define los parámetros para medir la fiabilidad de los patrones:")
display(change_percentage_input, periods_input, calculate_button, stats_output)


# ### Contexto e Interpretación de los Patrones

# - **Doji**: Un patrón de indecisión, donde la presión de compra y venta está equilibrada. A menudo precede a un cambio de dirección del mercado.
# - **Martillo (Hammer)**: Patrón de reversión alcista. Indica que el activo se vendió agresivamente, pero los compradores lograron empujar el precio al alza.
# - **Estrella Fugaz (Shooting Star)**: Patrón de reversión bajista. Sugiere que los compradores intentaron subir el precio, pero los vendedores tomaron el control y lo empujaron hacia abajo.
# - **Engulfing Alcista**: Patrón de reversión alcista fuerte. Una vela alcista grande envuelve por completo a la vela bajista anterior, mostrando un cambio de impulso.
# - **Engulfing Bajista**: Patrón de reversión bajista fuerte. Una vela bajista grande envuelve por completo a la vela alcista anterior, mostrando que los vendedores han tomado el control.
# - **Harami Alcista**: Patrón de reversión alcista. Una vela bajista grande es seguida por una vela alcista pequeña contenida en su cuerpo, sugiriendo una posible pérdida de impulso bajista.
# - **Harami Bajista**: Patrón de reversión bajista. Una vela alcista grande es seguida por una vela bajista pequeña contenida en su cuerpo, sugiriendo una posible pérdida de impulso alcista.
# - **Peonza (Spinning Top)**: Otro patrón de indecisión, similar al Doji pero con un cuerpo ligeramente más grande. Indica que ni compradores ni vendedores tienen el control.
# - **Doji Libélula**: Indica una fuerte reversión alcista, ya que el precio fue empujado hacia abajo pero los compradores lo llevaron de vuelta al precio de apertura/cierre.
# - **Doji Lápida**: Indica una fuerte reversión bajista, ya que los compradores no pudieron mantener el precio alto y los vendedores lo empujaron de vuelta al precio de apertura/cierre.
# - **Marubozu**: Indica un movimiento fuerte. Un Marubozu alcista sugiere un control total de los compradores, mientras que un Marubozu bajista sugiere un control total de los vendedores.


HTML(value='\n<style>\n    .header-container {\n        display: flex;\n        align-items: center;\n        …

Ingrese el símbolo y el rango de fechas:


Text(value='BTC-USD', description='Símbolo del activo:', placeholder='Ej: AAPL, TSLA, BTC-USD', style=Descript…

DatePicker(value=datetime.date(2025, 3, 23), description='Fecha de inicio:', style=DescriptionStyle(descriptio…

DatePicker(value=datetime.date(2025, 9, 19), description='Fecha de fin:', style=DescriptionStyle(description_w…

...

Define los parámetros para medir la fiabilidad de los patrones:


FloatText(value=2.0, description='Cambio porcentual (%):', style=DescriptionStyle(description_width='initial')…

IntText(value=10, description='Período (velas):', style=DescriptionStyle(description_width='initial'))

Button(button_style='info', description='Calcular Fiabilidad', icon='check', style=ButtonStyle())

Output()